In [1]:
import pandas as pd
import numpy as np
import gzip #to read gzip files
import matplotlib.pyplot as plt
import datetime
import pickle
from scipy.stats import linregress
import torch
import spacy 
#set the seed
np.random.seed(4)

DATA_PATH = './data/'
DEFAULT_ENCODING = 'UTF8'
DEFAULT_COMPRESSION = 'gzip'

/Users/loickreienbuhl/opt/anaconda3/envs/ADA_sentiment_analysis/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

matched_beers = pd.read_csv(DATA_PATH+'matched_beer_data/beers.csv', header=1)
matched_beers.head(2)

,abv,avg,avg_computed,avg_matched_valid_ratings,ba_score,beer_id,beer_name,beer_wout_brewery_name,brewery_id,brewery_name,...,brewery_id.1,brewery_name.1,nbr_matched_valid_ratings.1,nbr_ratings.1,overall_score,style.1,style_score,zscore.1,diff,sim
0,4.8,3.45,3.439867,3.504068,80.0,19827,Legbiter,Legbiter,10093,Strangford Lough Brewing Company Ltd,...,4959,Strangford Lough,89,89,23.0,Golden Ale/Blond Ale,27.0,-0.698304,1.000000,1.0
1,6.0,3.86,3.888750,3.768333,NaN,20841,St. Patrick's Ale,Patricks Ale St,10093,Strangford Lough Brewing Company Ltd,...,4959,Strangford Lough,11,11,52.0,Irish Ale,79.0,0.005754,0.527141,1.0


In [5]:
matched_users = pd.read_csv(DATA_PATH+'matched_beer_data/users.csv', header=1)
matched_users['joined'] = pd.to_datetime(matched_users['joined'],unit = 's')
matched_users['joined.1'] = pd.to_datetime(matched_users['joined.1'],unit = 's')
matched_users.head(2)

,joined,location,nbr_ratings,nbr_reviews,user_id,user_name,user_name_lower,joined.1,location.1,nbr_ratings.1,user_id.1,user_name.1,user_name_lower.1
0,2008-09-08 10:00:00,Germany,6,6,erzengel.248045,Erzengel,erzengel,2008-10-18 10:00:00,Germany,8781,83106,Erzengel,erzengel
1,2012-09-23 10:00:00,"United States, Virginia",1,1,gendv138.695700,GenDV138,gendv138,2013-11-29 11:00:00,"United States, Virginia",6240,290599,GenDV138,gendv138


Extracting matched data ratings

In [6]:
matched_ratings = pd.read_csv(DATA_PATH+'matched_beer_data/ratings.csv', encoding = "ISO-8859-1", header=1)
matched_ratings.head(2)

,abv,appearance,aroma,beer_id,beer_name,brewery_id,brewery_name,date,overall,palate,...,brewery_name.1,date.1,overall.1,palate.1,rating.1,style.1,taste.1,text.1,user_id.1,user_name.1
0,11.3,4.5,4.5,645,Trappistes Rochefort 10,207,Brasserie de Rochefort,1324810800,5.0,4.5,...,Brasserie Rochefort,1387710000,19.0,4.0,4.6,Abt/Quadrupel,9.0,a) Geruch malzig-schwer-sÃÂ¼ÃÂ. Riecht sc...,83106,Erzengel
1,5.0,NaN,NaN,28191,Myanmar Lager Beer,9369,Myanmar Brewery and Distillery,1322650800,NaN,NaN,...,Myanmar Brewery and Distillery,1322564400,6.0,2.0,1.7,Pale Lager,4.0,"Can. Weak and watery, not the best beer of the...",91324,visionthing


In [9]:
df_matched_BA = matched_ratings.iloc[:,:17]
df_matched_BA.set_axis(df_matched_BA.loc[0], axis='columns', inplace=False)
df_matched_BA = df_matched_BA.drop(0)
df_matched_BA.head(2)

/var/folders/kl/czv6zchx13lgg_2bc3mlk3h00000gn/T/ipykernel_15698/3821278892.py:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df_matched_BA.set_axis(df_matched_BA.loc[0], axis='columns', inplace=False)


,abv,appearance,aroma,beer_id,beer_name,brewery_id,brewery_name,date,overall,palate,rating,review,style,taste,text,user_id,user_name
1,5.0,NaN,NaN,28191,Myanmar Lager Beer,9369,Myanmar Brewery and Distillery,1322650800,NaN,NaN,3.00,True,American Adjunct Lager,NaN,NaN,visionthing.639993,visionthing
2,5.0,3.5,3.5,57911,Cantillon Tyrnilambic Baie DâArgousier Lambic,388,Brasserie Cantillon,1344074400,4.0,4.0,3.85,True,Lambic - Fruit,4.0,"Bottle @ One Pint Pub, Helsinki. 2006 vintage....",tiong.608427,tiong


In [10]:
df_matched_RB = matched_ratings.iloc[:,17:]
df_matched_RB.set_axis(df_matched_RB.loc[0], axis='columns', inplace=False)
df_matched_RB = df_matched_RB.drop(0)
df_matched_RB.head(2)

/var/folders/kl/czv6zchx13lgg_2bc3mlk3h00000gn/T/ipykernel_15698/988035139.py:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df_matched_RB.set_axis(df_matched_RB.loc[0], axis='columns', inplace=False)


,abv.1,appearance.1,aroma.1,beer_id.1,beer_name.1,brewery_id.1,brewery_name.1,date.1,overall.1,palate.1,rating.1,style.1,taste.1,text.1,user_id.1,user_name.1
1,5.0,2.0,3.0,17109,Myanmar Lager Beer,2921,Myanmar Brewery and Distillery,1322564400,6.0,2.0,1.7,Pale Lager,4.0,"Can. Weak and watery, not the best beer of the...",91324,visionthing
2,5.0,4.0,8.0,35298,Cantillon Tyrnilambic Baie dÂArgousier,1069,Cantillon,1353582000,17.0,4.0,4.1,Lambic Style - Fruit,8.0,"Bottle @ One Pint Pub, Helsinki. Originally ra...",98624,tiong


### Trying out Spacy

In [8]:
nlp = spacy.load("en_core_web_sm") # calling the effiency module of spacy
doc = nlp("I love cakes.")
print([(w.text, w.pos_) for w in doc])

[('I', 'PRON'), ('love', 'VERB'), ('cakes', 'NOUN'), ('.', 'PUNCT')]


In [9]:
with open('/Users/loickreienbuhl/Documents/EPFL/MA1/ADA/P3/DataframeStorage/adjectives_freq.pickle', 'rb') as file:
    adjective_freq = pickle.load(file)

AttributeError: Can't get attribute 'def_value' on <module '__main__'>